In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
keras.__version__

'2.5.0'

In [19]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

In [20]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
cgl = pd.read_json("../data/large_data/cgl.json")

In [4]:
cgl.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,...,lemmata,lemmata_wordcount,subcorpus,lemmata_repl,lemmatized_sentences_repl,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*,conc_lype
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...",71863,None,"[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελο...","[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ...",31,0,6,25,"[[μέγας, κινδυνεύοντας, δέχομαι, ἀείμνηστος, μ..."
6,tlg0006.tlg001.perseus-grc2.xml,Euripides,Cyclops,4141,tlg0006,tlg0006.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Βρόμιος, ἔχω, πόνος, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...",2535,None,"[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθενέ...","[[Βρόμιος, ἔχω, πόνο*, χὥτʼ, ἥβη, ἐμός, εὐσθεν...",7,0,0,1,"[[βοτόν, οὔτις, θύω, θεός, μέγας, γαστήρ, δαίμ..."
7,tlg0006.tlg004.perseus-grc2.xml,Euripides,Ἡρακλεῖδαι,6272,tlg0006,tlg0006.tlg004,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...",3545,None,"[ποτός, εἰμί, οὗτος, δεδογμένον, δίκαιος, φύω,...","[[ποτός, εἰμί, οὗτος, δεδογμένον], [δίκαιος, φ...",11,0,1,2,"[[εἰμί, πολύς, χαίρω, δυσφημέω, ἅζομαι, θέα, σ..."
8,tlg0006.tlg005.perseus-grc2.xml,Euripides,Ἱππόλυτος,8257,tlg0006,tlg0006.tlg005,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...",4898,None,"[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις, ...","[[πολύς, βροτός, ἀνώνυμος, θεά, καλέω, Κύπρις,...",8,3,8,8,"[[κοιτάζω, λέχος, σός, ναυβάτης, τὶς, πλέω, κρ..."
9,tlg0006.tlg006.perseus-grc2.xml,Euripides,Ἀνδρομάχη,7397,tlg0006,tlg0006.tlg006,5 B.C.,-4.5,{'-4.5': 1},-4.5,...,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...",4420,None,"[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πολ...","[[Ἀσιανός, γῆ, σχῆμα, θηβαία, πόλις, ἑδνόω, πο...",5,0,7,4,"[[Τροία, πράσσω, μηδείς, ὅδε, αὐχέω, πράσσω, δ..."


In [5]:
cgl[cgl["author_id"]=="tlg0086"]

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,...,lemmata,lemmata_wordcount,subcorpus,lemmata_repl,lemmatized_sentences_repl,count_πόνο*,count_ὀδύν*,count_ἄλγ*,count_λύπ*,conc_lype
685,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora,59614,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστή...",24680,c_aristotelicum,"[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστή...","[[πρῶτος, λέγω, εἰμί, σκέψις, ἀπόδειξις, ἐπιστ...",0,0,1,0,[]
686,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima,20912,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτε...",9395,c_aristotelicum,"[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτε...","[[καλός, τίμιος, εἶδος, ὑπολαμβάνω, ἕτερος, ἕτ...",0,0,0,12,"[[εἰμί, κινέω, ψυχή, οἷος, κινέω, τόπος, αὐτός..."
687,tlg0086.tlg003.perseus-grc1.xml,Aristotle,Athenian Constitution,16459,tlg0086,tlg0086.tlg003,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[Ἀθηναῖος, ἀρχή, χράω, βασίλειος, συνοικησάντο...",8088,None,"[Ἀθηναῖος, ἀρχή, χράω, βασίλειος, συνοικησάντο...","[[Ἀθηναῖος, ἀρχή, χράω, βασίλειος], [συνοικησά...",0,2,1,0,[]
688,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo,30794,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...",13437,c_aristotelicum,"[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγεθ...","[[φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶμα, μέγε...",1,0,0,0,[]
689,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae,10316,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...",4678,c_aristotelicum,"[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα, ...","[[ὁμώνυμος, λέγω, ὄνομα, μόνος, κοινός, ὄνομα,...",0,0,0,2,"[[σύμπτωμα, γίγνομαι, δυσαπάλλακτος, ἀκίνητος,..."
690,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum,1194,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐν...",549,c_aristotelicum,"[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐν...","[[μαντικός, ὕπνος, γίγνομαι, λέγω, συμβαίνω, ἐ...",0,0,0,0,[]
691,tlg0086.tlg009.perseus-grc1.xml,Aristotle,Eudemian Ethics (Greek). Machine readable text,26345,tlg0086,tlg0086.tlg009,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[δῆλος, θεός, ἑαυτοῦ, γνώμη, ἀποφηνάμενος, συγ...",11564,c_aristotelicum,"[δῆλος, θεός, ἑαυτοῦ, γνώμη, ἀποφηνάμενος, συγ...","[[δῆλος, θεός, ἑαυτοῦ, γνώμη, ἀποφηνάμενος, συ...",7,1,4,76,"[[αἵρεσις, δίδωμι, γίγνομαι, βίος, ζῶ, παῖς, ε..."
692,tlg0086.tlg010.perseus-grc1.xml,Aristotle,Nicomachean Ethics,56620,tlg0086,tlg0086.tlg010,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[τέχνη, πᾶς, μέθοδος, πρᾶξις, προαίρεσις, ἀγαθ...",25082,c_aristotelicum,"[τέχνη, πᾶς, μέθοδος, πρᾶξις, προαίρεσις, ἀγαθ...","[[τέχνη, πᾶς, μέθοδος, πρᾶξις, προαίρεσις, ἀγα...",11,0,6,153,"[[πολύς, γίγνομαι, μακάριος, βίος, ποιέω, συνε..."
693,tlg0086.tlg013.1st1K-grc2.xml,Aristotle,De generatione et corruptione,16914,tlg0086,tlg0086.tlg013,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[γένεσις, φθορά, φύσις, γίγνομαι, φθειρομένων,...",7430,None,"[γένεσις, φθορά, φύσις, γίγνομαι, φθειρομένων,...","[[γένεσις, φθορά, φύσις, γίγνομαι, φθειρομένων...",0,0,0,1,"[[κινούμενα, ὁμογενής, ἀνάγκη, δοκέω, εἰμί, ἁπ..."
694,tlg0086.tlg014.1st1K-grc1.xml,Aristotle,Historia animalium,93923,tlg0086,tlg0086.tlg014,4 B.C.,-3.5,{'-3.5': 1},-3.5,...,"[ζῷον, μόριον, εἰμί, ἀσύνθετος, ὅσος, διαιρέω,...",43967,c_aristotelicum,"[ζῷον, μόριον, εἰμί, ἀσύνθετος, ὅσος, διαιρέω,...","[[ζῷον, μόριον, εἰμί, ἀσύνθετος, ὅσος, διαιρέω...",40,0,12,5,"[[οὗτος, τείνω, σφαγή, ἑκάτερος, ὠμοπλάτη, τεί..."


In [6]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

In [7]:
aristotle_worklist = cgl[cgl["author_id"]=="tlg0086"]["clean_string"]

In [8]:
for text in aristotle_worklist:
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    print('Number of sequences:', len(sentences))

Number of sequences: 108684
Number of sequences: 148111
Number of sequences: 183369
Number of sequences: 241559
Number of sequences: 261452
Number of sequences: 263895
Number of sequences: 314582
Number of sequences: 425036
Number of sequences: 456879
Number of sequences: 641728
Number of sequences: 650749
Number of sequences: 655654
Number of sequences: 667471
Number of sequences: 670980
Number of sequences: 674349
Number of sequences: 688917
Number of sequences: 743318
Number of sequences: 748126
Number of sequences: 894454
Number of sequences: 997586
Number of sequences: 1010907
Number of sequences: 1089936
Number of sequences: 1192512
Number of sequences: 1212267
Number of sequences: 1346736
Number of sequences: 1358404
Number of sequences: 1444891
Number of sequences: 1472347
Number of sequences: 1487156
Number of sequences: 1493102
Number of sequences: 1500546
Number of sequences: 1586790
Number of sequences: 1589929
Number of sequences: 1594482
Number of sequences: 1598861


In [9]:
aristotle_string = " ".join(cgl[cgl["author_id"]=="tlg0086"]["clean_string"])

In [16]:
# List of unique characters in the corpus
chars = sorted(list(set(aristotle_string)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Unique characters: 263
Vectorization...


/var/folders/zg/zvg9y3rs7j527jxfq9sc2xqc0000gn/T/ipykernel_40858/776815168.py:9: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
/var/folders/zg/zvg9y3rs7j527jxfq9sc2xqc0000gn/T/ipykernel_40858/776815168.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences),

## Building the network

Our network is a single `LSTM` layer followed by a `Dense` classifier and softmax over all possible characters. But let us note that
recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in
recent times.

In [12]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Metal device set to: Apple M1


2022-07-12 11:02:14.579800: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-12 11:02:14.580049: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [13]:
#optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

## Training the language model and sampling from it


Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model,
and draw a character index from it (the "sampling function"):

In [14]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures
after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of
temperature in the sampling strategy.

In [17]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1


InvalidArgumentError: Value for attr 'T' of bool is not in the list of allowed values: half, float, double
	; NodeDef: {{node CudnnRNN}}; Op<name=CudnnRNN; signature=input:T, input_h:T, input_c:T, params:T -> output:T, output_h:T, output_c:T, reserve_space:T; attr=T:type,allowed=[DT_HALF, DT_FLOAT, DT_DOUBLE]; attr=rnn_mode:string,default="lstm",allowed=["rnn_relu", "rnn_tanh", "lstm", "gru"]; attr=input_mode:string,default="linear_input",allowed=["linear_input", "skip_input", "auto_select"]; attr=direction:string,default="unidirectional",allowed=["unidirectional", "bidirectional"]; attr=dropout:float,default=0; attr=seed:int,default=0; attr=seed2:int,default=0; attr=is_training:bool,default=true; is_stateful=true> [Op:CudnnRNN]


As you can see, a low temperature results in extremely repetitive and predictable text, but where local structure is highly realistic: in
particular, all words (a word being a local pattern of characters) are real English words. With higher temperatures, the generated text
becomes more interesting, surprising, even creative; it may sometimes invent completely new words that sound somewhat plausible (such as
"eterned" or "troveration"). With a high temperature, the local structure starts breaking down and most words look like semi-random strings
of characters. Without a doubt, here 0.5 is the most interesting temperature for text generation in this specific setup. Always experiment
with multiple sampling strategies! A clever balance between learned structure and randomness is what makes generation interesting.

Note that by training a bigger model, longer, on more data, you can achieve generated samples that will look much more coherent and
realistic than ours. But of course, don't expect to ever generate any meaningful text, other than by random chance: all we are doing is
sampling data from a statistical model of which characters come after which characters. Language is a communication channel, and there is
a distinction between what communications are about, and the statistical structure of the messages in which communications are encoded. To
evidence this distinction, here is a thought experiment: what if human language did a better job at compressing communications, much like
our computers do with most of our digital communications? Then language would be no less meaningful, yet it would lack any intrinsic
statistical structure, thus making it impossible to learn a language model like we just did.


## Take aways

* We can generate discrete sequence data by training a model to predict the next tokens(s) given previous tokens.
* In the case of text, such a model is called a "language model" and could be based on either words or characters.
* Sampling the next token requires balance between adhering to what the model judges likely, and introducing randomness.
* One way to handle this is the notion of _softmax temperature_. Always experiment with different temperatures to find the "right" one.